In [1]:
!pip3 install -U sentence-transformers
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.7 MB/s 
     |████████████████████████████████| 4.4 MB 57.4 MB/s 
     |████████████████████████████████| 1.2 MB 63.8 MB/s 
     |████████████████████████████████| 101 kB 11.2 MB/s 
     |████████████████████████████████| 596 kB 72.3 MB/s 
     |████████████████████████████████| 6.6 MB 17.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=625bed0f5cf3df7200ffce151eb054ae23ec70be4eac44a710b2661965dfd3cc
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from sentence_transformers.datasets import DenoisingAutoEncoderDataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models
from sentence_transformers.losses import DenoisingAutoEncoderLoss

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file= "/content/drive/MyDrive/Sentence-Transformer-Base/tsdae_train.txt"

In [5]:
with open(file) as f:
  sentences = [item.rstrip('\n') for item in f.readlines()]
sentences[0:5]

['scala log version output remov osgi depend directori tycho build project add f refactor warn get classpath string util overrid addit file element back',
 'scala log version osgi depend tycho project add f except compil refactor warn get classpath string util overrid addit file element set back',
 'scala log version output remov osgi depend directori tycho build project add f except compil refactor warn get classpath string util overrid addit file element set back',
 'scala log version output remov depend directori build project add f except compil refactor warn get classpath string util overrid addit file element set back',
 'log output failur directori analysi sourc canon except execut compil use ignor mode debug get classpath dir util overrid path info cach enabl file element set goal nb']

In [6]:
# dataset class with noise functionality built-in
train_data = DenoisingAutoEncoderDataset(sentences)

# we use a dataloader as usual
loader = DataLoader(train_data, batch_size=8, shuffle=True, drop_last=True)


bert = models.Transformer('bert-base-uncased')
pooling = models.Pooling(bert.get_word_embedding_dimension(), 'cls')

model = SentenceTransformer(modules=[bert, pooling], device='cuda')
model

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

        :param train_objectives: Tuples of (DataLoader, LossFunction). Pass more than one for multi-task learning
        :param evaluator: An evaluator (sentence_transformers.evaluation) evaluates the model performance during training on held-out dev data. It is used to determine the best model that is saved to disc.
        :param epochs: Number of epochs for training
        :param steps_per_epoch: Number of training steps per epoch. If set to None (default), one epoch is equal the DataLoader size from train_objectives.
        :param scheduler: Learning rate scheduler. Available schedulers: constantlr, warmupconstant, warmuplinear, warmupcosine, warmupcosinewithhardrestarts
        :param warmup_steps: Behavior depends on the scheduler. For WarmupLinear (default), the learning rate is increased from o up to the maximal learning rate. After these many training steps, the learning rate is decreased linearly back to zero.
        :param optimizer_class: Optimizer
        :param optimizer_params: Optimizer parameters
        :param weight_decay: Weight decay for model parameters
        :param evaluation_steps: If > 0, evaluate the model using evaluator after each number of training steps
        :param output_path: Storage path for the model and evaluation files
        :param save_best_model: If true, the best model (according to evaluator) is stored at output_path
        :param max_grad_norm: Used for gradient normalization.
        :param use_amp: Use Automatic Mixed Precision (AMP). Only for Pytorch >= 1.6.0
        :param callback: Callback function that is invoked after each evaluation.
                It must accept the following three parameters in this order:
                `score`, `epoch`, `steps`
        :param show_progress_bar: If True, output a tqdm progress bar
        :param checkpoint_path: Folder to save checkpoints during training
        :param checkpoint_save_steps: Will save a checkpoint after so many steps
        :param checkpoint_save_total_limit: Total number of checkpoints to store

In [7]:
# configuration 1
loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)
model.fit(
    train_objectives=[(loader, loss)],
    epochs=4,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model.save('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.11.crossattention.self.query.weight', 'bert.encoder.layer.8.crossattention.self.query.weight', 'bert.encoder.layer.6.crossattention.self.value.weight', 'bert.enco

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3481 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3481 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3481 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3481 [00:00<?, ?it/s]

In [ ]:
# configuration 2
loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)
model.fit(
    train_objectives=[(loader, loss)],
    epochs=4,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 1e-5},
    show_progress_bar=True
)

model.save('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased-conf2')

Some weights of BertLMHeadModel were not initialized from the model checkpoint at /content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased/ and are newly initialized: ['encoder.layer.11.crossattention.self.query.weight', 'encoder.layer.3.crossattention.self.query.bias', 'encoder.layer.8.crossattention.self.query.weight', 'encoder.layer.3.crossattention.self.value.bias', 'encoder.layer.8.crossattention.output.dense.weight', 'encoder.layer.8.crossattention.self.key.weight', 'encoder.layer.4.crossattention.output.dense.weight', 'encoder.layer.5.crossattention.output.dense.weight', 'encoder.layer.11.crossattention.self.key.bias', 'encoder.layer.0.crossattention.self.key.weight', 'encoder.layer.11.crossattention.self.value.bias', 'encoder.layer.0.crossattention.output.dense.bias', 'encoder.layer.9.crossattention.output.LayerNorm.bias', 'encoder.layer.1.crossattention.output.LayerNorm.weight', 'encoder.layer.6.crossattention.output.LayerNorm.bias', 'encoder.layer.5.crossatten

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

In [ ]:
# configuration 3
loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)
model.fit(
    train_objectives=[(loader, loss)],
    epochs=4,
    weight_decay=0,
    scheduler='warmupcosine',
    optimizer_params={'lr': 1e-5},
    show_progress_bar=True
)

model.save('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased-conf3')

Some weights of BertLMHeadModel were not initialized from the model checkpoint at /content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased-conf2/ and are newly initialized: ['encoder.layer.11.crossattention.self.query.weight', 'encoder.layer.3.crossattention.self.query.bias', 'encoder.layer.8.crossattention.self.query.weight', 'encoder.layer.3.crossattention.self.value.bias', 'encoder.layer.8.crossattention.output.dense.weight', 'encoder.layer.8.crossattention.self.key.weight', 'encoder.layer.4.crossattention.output.dense.weight', 'encoder.layer.5.crossattention.output.dense.weight', 'encoder.layer.11.crossattention.self.key.bias', 'encoder.layer.0.crossattention.self.key.weight', 'encoder.layer.11.crossattention.self.value.bias', 'encoder.layer.0.crossattention.output.dense.bias', 'encoder.layer.9.crossattention.output.LayerNorm.bias', 'encoder.layer.1.crossattention.output.LayerNorm.weight', 'encoder.layer.6.crossattention.output.LayerNorm.bias', 'encoder.layer.5.cros

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

In [7]:
# configuration 4
loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)
model.fit(
    train_objectives=[(loader, loss)],
    epochs=4,
    weight_decay=0,
    scheduler='warmuplinear',
    optimizer_params={'lr': 1e-5},
    show_progress_bar=True
)

model.save('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased-conf4')



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.6.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.11.crossattention.self.query.bias', 'bert.encoder.layer.5.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'ber

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3481 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3481 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3481 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3481 [00:00<?, ?it/s]

In [ ]:
# configuration 5
loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)
model.fit(
    train_objectives=[(loader, loss)],
    epochs=4,
    weight_decay=0,
    scheduler='warmupcosine',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model.save('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased-conf5')

In [ ]:
# configuration 6
loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)
model.fit(
    train_objectives=[(loader, loss)],
    epochs=4,
    weight_decay=0,
    scheduler='warmuplinear',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model.save('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased-conf6')



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.3.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.11.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.10.crossattention.output.LayerNorm.bias

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3985 [00:00<?, ?it/s]

In [4]:
model = SentenceTransformer('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased')

In [ ]:
model = SentenceTransformer('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased-conf2')

In [ ]:
model = SentenceTransformer('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased-conf3')

In [8]:
model = SentenceTransformer('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased-conf4')

In [ ]:
model = SentenceTransformer('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased-conf5')

In [ ]:
model = SentenceTransformer('/content/drive/MyDrive/Sentence-Transformer-Base/tsdae-bert-base-uncased-conf6')

READ eval set to find the best configuration that must be used on the test set

In [9]:
file_test= "/content/drive/MyDrive/Sentence-Transformer-Base/tsdae_eval.txt"
f=open(file_test)
lines=f.readlines()
lines=[l.strip() for l in lines]
f.close()

In [10]:
lines[0:2]

['[\'44|1079|0\', \'1644\', "[\'split\', \'read\', \'buffer\', \'reset\', \'min\', \'fulli\', \'skip\', \'valid\', \'buf\', \'reader\', \'char\', \'max\', \'ahead\', \'true\', \'mark\', \'length\', \'len\', \'po\', \'offset\', \'point\']", "{144: [\'bug\', \'fix\', \'issu\'], 173: [\'method\', \'elemnet\', \'string\', \'add\'], 226: [\'default\', \'set\', \'charset\'], 228: [\'featur\', \'encod\', \'add\', \'connect\', \'data\', \'org\', \'jsoup\'], 324: [\'visitor\', \'visit\', \'selector\', \'node\', \'add\', \'remov\'], 340: [\'android\', \'connect\', \'char\', \'redirect\'], 363: [\'file\', \'support\', \'test\'], 369: [\'request\', \'abil\', \'chang\', \'charset\'], 371: [\'tag\', \'support\', \'refresh\', \'redirect\', \'meta\', \'html\'], 381: [\'soup\', \'fix\', \'cach\', \'ad\', \'connect\', \'j\', \'use\', \'paramet\'], 384: [\'navig\', \'style\', \'dom\', \'add\'], 448: [\'restructur\', \'studio\', \'gradl\'], 502: [\'configur\', \'close\', \'tag\'], 576: [\'whitespac\', \'e

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from statistics import mean

ranks=list()
rank_best_pred=list()

for l in lines:
  line=eval(l)
  ok_issue=int(line[1])
  method_tokens=" ".join(eval(line[2]))
  dict_open=eval(line[3])
  issues=list()
  issues_id=list()
  for k in dict_open.keys():
    issues_id.append(int(k))
    issues.append(" ".join(dict_open[k]))
  
  issue_embeddings = model.encode(issues)
  print(issue_embeddings.shape)

  print(method_tokens)

  method_embedding=model.encode([method_tokens])
  simil=cosine_similarity(
    [method_embedding[0]],
    issue_embeddings
  )
  simil=simil[0]
  print(simil)
  order=sorted(range(len(simil)), key=lambda k: simil[k], reverse=True)
  print(order)

  best_issue_id=order[0]
  best_issue=issues[best_issue_id]

  print(issues_id)
  print(ok_issue)
  correct_position = issues_id.index(ok_issue)
  print(correct_position)

  rank=order.index(correct_position)
  print(rank)
  ranks.append(1/(rank+1))
  rank_best_pred.append(rank+1)

print(mean(ranks))
counter=Counter(rank_best_pred)

counter=dict(sorted(counter.items()))
print(counter)

Streaming output truncated to the last 5000 lines.
 0.5953788  0.5549029  0.5463283  0.5769423  0.47261584]
[12, 10, 7, 15, 2, 13, 14, 9, 4, 5, 3, 1, 11, 16, 8, 6, 0]
[41, 42, 75, 76, 91, 94, 97, 107, 122, 123, 133, 144, 145, 164, 168, 169, 170]
164
13
5
(73, 768)
control lifecycl builder client open assist shift extens observ cube registrar creator suit regist overrid
[0.7023227  0.4929399  0.49978507 0.57169676 0.6108201  0.6023944
 0.56947774 0.6321144  0.70918196 0.69369256 0.5764941  0.6323684
 0.55671656 0.5239389  0.6888877  0.5815216  0.5965721  0.72874475
 0.583559   0.5404227  0.6228283  0.5927021  0.6126613  0.61840713
 0.6402045  0.5674886  0.6649211  0.6631137  0.5763027  0.6088479
 0.5733728  0.6142504  0.5031733  0.5673979  0.5794331  0.6248375
 0.6571052  0.65461993 0.6267614  0.5659065  0.6069894  0.6216103
 0.65560544 0.5106148  0.64454556 0.64689267 0.654322   0.5860729
 0.72089624 0.5170424  0.61915714 0.6378668  0.5628694  0.53529286
 0.5663896  0.6030339  0.607872

EVALUATE THE BEST CONFIGURATION ON THE TEST SET

In [5]:
file_test= "/content/drive/MyDrive/Sentence-Transformer-Base/tsdae_test.txt"
f=open(file_test)
lines=f.readlines()
lines=[l.strip() for l in lines]
f.close()

In [6]:
lines[0:2]

['[\'0|18|0\', \'583\', "[\'scala\', \'scalac\', \'array\', \'split\', \'version\', \'plugin\', \'number\', \'releas\', \'target\', \'add\', \'list\', \'except\', \'collect\', \'compil\', \'arg\', \'get\', \'major\', \'string\', \'util\', \'find\', \'option\', \'empti\', \'minor\']", "{46: [\'gener\', \'pick\', \'sourc\', \'scaladoc\'], 53: [\'reload\', \'replay\', \'class\', \'consol\'], 71: [\'mvn\', \'leav\', \'consol\', \'exit\'], 102: [\'loop\', \'scala\', \'cc\', \'file\', \'chang\'], 111: [\'scala\', \'cc\', \'fsc\', \'hang\', \'window\', \'start\', \'use\'], 172: [\'work\', \'scala\', \'mvn\', \'cc\'], 177: [\'cross\', \'way\', \'compil\'], 179: [\'scala\', \'error\', \'complain\', \'cc\', \'mvn\', \'work\', \'compil\', \'test\'], 189: [\'detail\', \'scala\', \'compil\', \'use\'], 193: [\'packag\', \'gener\', \'scaladoc\', \'object\'], 476: [\'plugin\', \'increas\', \'build\', \'time\'], 480: [\'version\', \'consid\'], 492: [\'support\', \'option\', \'output\', \'configur\', \'

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from statistics import mean

ranks=list()
rank_best_pred=list()

for l in lines:
  line=eval(l)
  ok_issue=int(line[1])
  method_tokens=" ".join(eval(line[2]))
  dict_open=eval(line[3])
  issues=list()
  issues_id=list()
  for k in dict_open.keys():
    issues_id.append(int(k))
    issues.append(" ".join(dict_open[k]))
  
  issue_embeddings = model.encode(issues)
  print(issue_embeddings.shape)

  print(method_tokens)

  method_embedding=model.encode([method_tokens])
  simil=cosine_similarity(
    [method_embedding[0]],
    issue_embeddings
  )
  simil=simil[0]
  print(simil)
  order=sorted(range(len(simil)), key=lambda k: simil[k], reverse=True)
  print(order)

  best_issue_id=order[0]
  best_issue=issues[best_issue_id]

  print(issues_id)
  print(ok_issue)
  correct_position = issues_id.index(ok_issue)
  print(correct_position)

  rank=order.index(correct_position)
  print(rank)
  ranks.append(1/(rank+1))
  rank_best_pred.append(rank+1)

print(mean(ranks))
counter=Counter(rank_best_pred)

counter=dict(sorted(counter.items()))
print(counter)

(22, 768)
scala scalac array split version plugin number releas target add list except collect compil arg get major string util find option empti minor
[0.59200346 0.46101907 0.5424302  0.4923172  0.46314684 0.51868594
 0.53859144 0.54921734 0.605833   0.6092012  0.49897367 0.55957127
 0.5634838  0.5787852  0.72294986 0.56788915 0.59413236 0.41319484
 0.6319725  0.5516953  0.6387823  0.49016744]
[14, 20, 18, 9, 8, 16, 0, 13, 15, 12, 11, 19, 7, 2, 6, 5, 10, 3, 21, 4, 1, 17]
[46, 53, 71, 102, 111, 172, 177, 179, 189, 193, 476, 480, 492, 495, 499, 502, 507, 514, 553, 558, 575, 583]
583
21
18
(22, 768)
scala scalac array split version plugin number releas target add list except collect compil arg get major string util find option empti minor
[0.59200346 0.46101907 0.5424302  0.4923172  0.46314684 0.51868594
 0.53859144 0.54921734 0.605833   0.6092012  0.49897367 0.55957127
 0.5634838  0.5787852  0.72294986 0.56788915 0.59413236 0.41319484
 0.6319725  0.5516953  0.6387823  0.49016744]
[14, 